In [20]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import calendar
import json
import csv
import numpy as np
from selenium import webdriver
import time
import math
import re
import datetime

# Getting local attributes

In this section we want to grab all the informaion from their websites. It is important to mention that there will be some cases where the link that we generated does not work, we should handle those cases seperately.  \
The most information that we can grab is:
- Address
- Telephone number
- Prices of different fuel types (it will be convenient to put these in a dictionary and later separate them)
- Services
- Opening Hours

Depening on your internet it will run for about 4-5 hours.

In [2]:
gas_stations = pd.read_csv('gas_stations_with_link.csv')

In [ ]:
from selenium.webdriver.chrome.options import Options
import warnings
warnings.filterwarnings('ignore', category = DeprecationWarning)

chrome_options = Options()
chrome_options.add_argument('--headless')

def grabbing_local_attributes(link):
    '''
    This function returns a list of attributes which we are going to seperate afterwards. It is important to mention to try-except cycle seperately since not every link contains all the information.
    '''
    
    driver = webdriver.Chrome('/home/csaba/Downloads/chromedriver', options=chrome_options)
    driver.get(link)
    time.sleep(4)
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    
    local_attributes = []
    elements = driver.find_elements(By.CLASS_NAME, "card-body")
    
    try: #address and telephone
        local_attributes.append(elements[0].text.replace('\n', ' ').replace(',', ';'))
    except:
        local_attributes.append('ERROR')
        
    try: #gas prices
        local_attributes.append(elements[1].text.replace('\n', ' ').replace(',', ';'))
    except:
        local_attributes.append('ERROR')

    try: #services
        local_attributes.append(elements[2].text.replace('\n', ' ').replace(',', ';'))
    except:
        local_attributes.append('ERROR')
        
    try: #opening hours
        local_attributes.append(elements[3].text.replace('\n', ' ').replace(',', ';'))
    except:
        local_attributes.append('ERROR')
    
    #print(local_attributes)
    return (local_attributes[0], local_attributes[1], local_attributes[2], local_attributes[3])

gas_stations['Local Attributes'] = gas_stations['Link'].apply(grabbing_local_attributes)
gas_stations.to_csv('Gas_stations_march_28.csv', index=False,sep = '\t')

# Data Formatting and Cleaning

### Seperating the local attributes

In [21]:
gas_stations = pd.read_csv('Gas_stations_march_28.csv', sep = '\t')

#convert from string back to tuple
gas_stations['Local Attributes'] = gas_stations['Local Attributes'].apply(eval)

#remove the 4 tuple element and put them into seperate columns
gas_stations[['Address and Telephone', 'Gas Prices', 'Services', 'Opening Hours']] = gas_stations['Local Attributes'].apply(pd.Series)

gas_stations['Address from website'] = gas_stations['Address and Telephone'].apply(lambda x: x.split('Küldés telefonra:')[0].split('Útvonal:')[1].lstrip().replace(';', ',').rstrip() if 'Küldés telefonra:' in x else x)
gas_stations['Telephone'] = gas_stations['Address and Telephone'].apply(lambda x: re.sub('[^0-9]', '', x.split('Küldés telefonra:')[1]) if 'Küldés telefonra:' in x else x)
gas_stations['Gas Prices'] = gas_stations['Gas Prices'].apply(lambda x: x.split('!')[1] if '!' in x else x)

#dropping the separated columns
gas_stations.drop(['Local Attributes','Address and Telephone'], axis=1, inplace=True)

gas_stations = gas_stations.loc[~gas_stations['Opening Hours'].str.contains('ft|Ft')]

criteria = (~gas_stations['Gas Prices'].str.contains('Ft|Nem|nem')) & (gas_stations['Gas Prices'] != '')

gas_stations.loc[criteria, 'Gas Prices'] = gas_stations.loc[criteria, 'Services']
gas_stations.loc[criteria, 'Services'] = gas_stations.loc[criteria, 'Opening Hours']
gas_stations.loc[criteria, 'Opening Hours'] = 'ERROR'

gas_stations.loc[gas_stations['Services'].str.contains('ft|Ft'), 'Gas Prices'] = gas_stations.loc[gas_stations['Services'].str.contains('ft|Ft'), 'Services']
gas_stations.loc[gas_stations['Services'].str.contains('ft|Ft'), 'Services'] = gas_stations.loc[gas_stations['Services'].str.contains('ft|Ft'), 'Opening Hours']
gas_stations.loc[gas_stations['Services'].str.contains('ft|Ft'), 'Opening Hours'] = 'ERROR'

gas_stations['Gas Prices'] = gas_stations['Gas Prices'].apply(lambda x: x.split('!')[1] if '!' in x else x)
gas_stations.loc[~gas_stations['Opening Hours'].str.contains('Hétfő|ERROR'),'Opening Hours'] = 'ERROR'
gas_stations = gas_stations.drop('Opening Hours', axis=1) #we should drop it, it contains too many errors-

gas_stations['Telephone'] = '+36' + gas_stations['Telephone']
gas_stations['Telephone'] = gas_stations['Telephone'].apply(lambda x: x[:11] if len(x) > 13 else x)
gas_stations = gas_stations.drop('Telephone', axis=1) #in this case we drop it, we will not need it

### Cleaning Gas Prices

In [22]:
#we need to remove the date + 'óta aktuális' parts. Example: '2000-01-01 óta aktuális' -> replace with ''
gas_stations = gas_stations.reset_index(drop = True)

original_names = gas_stations.columns

dt = datetime.datetime(2000, 1, 1)
end = datetime.datetime(2023, 7, 31)
step = datetime.timedelta(days=1)

dates_to_remove = []

while dt < end:
    dates_to_remove.append(dt.strftime('%Y-%m-%d') + ' óta aktuális')
    dt += step

for date in dates_to_remove:
    gas_stations['Gas Prices'] = gas_stations['Gas Prices'].str.replace(date, '')
        
#we can separate with 'Ft /liter' and 'ismert' so the remaining part would only be category + price or category + nem which denotes unknown in hungarian
gas_stations['Gas Prices'] = gas_stations['Gas Prices'].str.split('Ft /liter|ismert|Ft /kw')
gas_stations['Gas Prices'] = gas_stations['Gas Prices'].apply(lambda x: [i for i in x if i not in ['', ' ']])
gas_stations['Gas Prices'] = gas_stations['Gas Prices'].apply(lambda x: [i.lstrip().rstrip() for i in x])

#now we have a column containing list elements for each, example:
#['Prémium Gázolaj 661.0;-', '95-ös Benzin E10 606.0;-', 'Gázolaj 606.0;-', 'Lpg 353.0;-']. We should separate by whitespaces, then a -1 element is the price, the others are the category
def convert_list_to_dict(l):
    
    output_dict = {}
    
    if l == []:
        return {}
    if l == ['ERROR']:
        return {}
    for item in l:
        key, value = item.rsplit(' ', 1)
        output_dict[key] = value
    return output_dict

gas_stations['Gas Prices'] = gas_stations['Gas Prices'].apply(convert_list_to_dict)
wanted_columns = ['95-ös Benzin E10', 'Gázolaj', 'Lpg', '100-As Benzin E5', 'Cng', 'Prémium Gázolaj', 'Prémium Benzin E10', '98-As Benzin E5', 'Adblue', 'Type-1 Elektromos Töltő', 'Type-2 Elektromos Töltő', 'Chademo Elektromos Töltő', 'Ccs - Elektromos Töltő']

all_cols = set().union(*gas_stations['Gas Prices'])

separated_variables = pd.DataFrame([{k: d.get(k) for k in wanted_columns} for d in gas_stations['Gas Prices']])
gas_stations.drop(['Gas Prices'], axis=1, inplace=True)

gas_stations = pd.concat([gas_stations, separated_variables], axis = 1)

gas_stations[wanted_columns] = gas_stations[wanted_columns].replace({'ERROR':np.nan, None:np.nan, 'Nem':np.nan})
gas_stations[wanted_columns] = gas_stations[wanted_columns].applymap(lambda x: x.replace(';-', '') if pd.notna(x) else x)
gas_stations[wanted_columns] = gas_stations[wanted_columns].astype(float)

### Cleanig Franchise variable

In [23]:
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('Avia', 'AVIA')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('avia', 'AVIA')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('lukoil', 'Lukoil')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('mobil_gaz_kft', 'Mobil Gáz')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('mobil-petrol', 'Mobil Petrol')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('mobil_petrol', 'Mobil Petrol')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('mol', 'MOL')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('MOL Partner', 'MOL')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('super_mol_kft_', 'MOL')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('mol-partner', 'MOL')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('MOL Partner+', 'MOL')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('MOL partner +', 'MOL')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('Mol Postaautó', 'MOL')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('OIL! Kiskunfélegyháza', 'OIL!')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('Oil Dunavarsány', 'OIL!')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('oil', 'OIL!')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('oil!', 'OIL!')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('omv', 'OMV')
gas_stations['Franchise (or private)'] = gas_stations['Franchise (or private)'].replace('shell', 'Shell')

franc = ['AVIA', 'Lukoil', 'Mobil Gáz', 'Mobil Petrol', 'MOL', 'OIL!', 'OMV', 'Shell']
gas_stations['Franchise (or private)']= np.where(gas_stations['Franchise (or private)'].isin(franc), gas_stations['Franchise (or private)'], 'Magán')

# Connecting the DataFrame with competition

In [24]:
competitions = pd.read_csv('competitions.csv', sep = '\t')
competitions

,City,1km,Number of competition 1km,5km,Number of competition 5km,10km,Number of competition 10km,20km,Number of competition 20km,30km,Number of competition 30km,40km,Number of competition 40km
0,Abda,"['Abda, Bécsi út 186.']",1,"['Abda, Bécsi út 186.', 'Abda, Bécsi u. 1.']",2,"['Abda, Bécsi út 186.', 'Abda, Bécsi u. 1.', '...",24,"['Abda, Bécsi út 186.', 'Abda, Bécsi u. 1.', '...",37,"['Abda, Bécsi út 186.', 'Abda, Bécsi u. 1.', '...",53,"['Abda, Bécsi út 186.', 'Abda, Bécsi u. 1.', '...",68
1,Abony,"['Abony, Radák u. ']",1,"['Abony, Radák u. ', 'Abony, Kécskei út 2/A']",2,"['Abony, Radák u. ', 'Abony, Kécskei út 2/A']",2,"['Abony, Radák u. ', 'Abony, Kécskei út 2/A',...",27,"['Abony, Radák u. ', 'Abony, Kécskei út 2/A',...",35,"['Abony, Radák u. ', 'Abony, Kécskei út 2/A',...",79
2,Ács,"['Ács, Fő út 2.']",1,"['Ács, Fő út 2.', 'Bábolna, Bábolna - Concó pi...",5,"['Ács, Fő út 2.', 'Bábolna, Bábolna - Concó pi...",10,"['Ács, Fő út 2.', 'Bábolna, Bábolna - Concó pi...",15,"['Ács, Fő út 2.', 'Bábolna, Bábolna - Concó pi...",59,"['Ács, Fő út 2.', 'Bábolna, Bábolna - Concó pi...",79
3,Adony,"['Adony, 6.sz. fkl. út ']",1,"['Adony, 6.sz. fkl. út ', 'Iváncsa, Hunyadi u...",2,"['Ráckeve, Kossuth L. u. ', 'Adony, 6.sz. fkl...",3,"['Iváncsa, Hunyadi utca 782/1.', 'Ráckeve, Kos...",25,"['Iváncsa, Hunyadi utca 782/1.', 'Ráckeve, Kos...",79,"['Iváncsa, Hunyadi utca 782/1.', 'Ráckeve, Kos...",236
4,Aggtelek,"['Aggtelek, Kossuth L. u. hrsz 1/2']",1,"['Aggtelek, Kossuth L. u. hrsz 1/2']",1,"['Aggtelek, Kossuth L. u. hrsz 1/2']",1,"['Aggtelek, Kossuth L. u. hrsz 1/2', 'Putnok, ...",3,"['Aggtelek, Kossuth L. u. hrsz 1/2', 'Putnok, ...",13,"['Aggtelek, Kossuth L. u. hrsz 1/2', 'Putnok, ...",22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,Zamárdi,"['Zamárdi, Endrédi út ']",1,"['Zamárdi, Endrédi út ', 'Siófok, Töreki út 1...",3,"['Zamárdi, Endrédi út ', 'Siófok, Töreki út 1...",12,"['Zamárdi, Endrédi út ', 'Siófok, Töreki út 1...",27,"['Zamárdi, Endrédi út ', 'Siófok, Töreki út 1...",52,"['Zamárdi, Endrédi út ', 'Siófok, Töreki út 1...",92
509,Zirc,"['Zirc, Kossuth Lajos út ']",1,"['Zirc, Kossuth Lajos út ', 'Zirc, Állomás ut...",2,"['Zirc, Kossuth Lajos út ', 'Zirc, Állomás ut...",3,"['Zirc, Kossuth Lajos út ', 'Zirc, Állomás ut...",19,"['Zirc, Kossuth Lajos út ', 'Zirc, Állomás ut...",40,"['Zirc, Kossuth Lajos út ', 'Zirc, Állomás ut...",79
510,Zomba,"['Zomba, Külterület 03/2.']",1,"['Zomba, Külterület 03/2.']",1,"['Zomba, Külterület 03/2.']",1,"['Zomba, Külterület 03/2.', 'Szekszárd, M6 I. ...",17,"['Zomba, Külterület 03/2.', 'Szekszárd, M6 I. ...",46,"['Zomba, Külterület 03/2.', 'Szekszárd, M6 I. ...",81
511,Zsámbék,"['Zsámbék, Herceghalmi út']",1,"['Zsámbék, Herceghalmi út', 'Zsámbék, M1 autóp...",3,"['Zsámbék, Herceghalmi út', 'Zsámbék, M1 autóp...",9,"['Zsámbék, Herceghalmi út', 'Zsámbék, M1 autóp...",55,"['Zsámbék, Herceghalmi út', 'Zsámbék, M1 autóp...",230,"['Zsámbék, Herceghalmi út', 'Zsámbék, M1 autóp...",343


In [25]:
gas_stations = pd.merge(gas_stations, competitions, on='City')

# Exporting the final dataframe

In [19]:
gas_stations.to_csv('Gas_Stations_03_28.csv', index = False, header = True)